In [ ]:
import serial
import time

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import serial.tools.list_ports
from collections import deque
from IPython.display import display, clear_output
from scipy.interpolate import interp1d

In [7]:
#what are we looking at
vmax = 5 # FSR
components = 'x' # 'xy'

# Live Reading

In [8]:
# List available ports and print them for user reference
ports = list(serial.tools.list_ports.comports())
print("Available ports:")
for port in ports:
    print(port.device)

Available ports:
COM5


In [9]:
# Set up the serial connection (update the port as needed)
serial_port = "COM5" 
baud_rate = 115200
window_size = 3750  # Number of samples to show on the plot
refresh_rate = 3750  # Update plot every this many samples


In [10]:
# Initialize data buffers
B_data = deque([0] * window_size, maxlen=window_size)
time_data = deque([0] * window_size, maxlen=window_size)

# Initialize plots
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))

# Plot for time-domain signal
line1, = ax1.plot(time_data, B_data)
ax1.set_xlabel("Time (s)")
ax1.set_ylabel("Voltage (V)")
#ax1.set_ylim(-0.5, 0.5)
ax1.grid(True) 

# Plot for FFT power spectrum
fft_line, = ax2.plot([], [], label="FFT Spectrum")
ax2.set_xscale('log')
ax2.set_yscale('log')
ax2.set_xlim(1, 500)  # Placeholder limits
ax2.set_xlabel("Frequency (Hz)")
ax2.set_ylabel("Power")
ax2.axvline(60, color='red', linestyle='--', linewidth=1, label="60 Hz")
ax2.axvline(180, color='red', linestyle='--', linewidth=1, label="60 Hz")
ax2.grid(True)
plt.ion()  # Enable interactive mode

def try_connect(serial_port, baud_rate):
    """Try to connect to the serial port."""
    while True:
        try:
            ser = serial.Serial(serial_port, baud_rate, timeout=1)
            print(f"Reconnected to {serial_port}")
            return ser
        except (serial.SerialException, OSError):
            print(f"Waiting for device on {serial_port}...")
            time.sleep(1)

try:
    ser = try_connect(serial_port, baud_rate)
    sample_count = 0

    while True:
        try:
            if ser.in_waiting:
                data_chunk = ser.read(ser.in_waiting).decode('utf-8')
                lines = data_chunk.splitlines()

                for line in lines:
                    # Parse voltage and time from the serial input
                    try:
                        voltage, timestamp_us = line.split(",")
                        voltage = float(voltage) / (2**23) * vmax  # Convert raw data to voltage
                        timestamp_s = float(timestamp_us) * 1e-6  # Convert microseconds to seconds
                        B_data.append(voltage)
                        time_data.append(timestamp_s)
                        sample_count += 1

                        # Update the plots every `refresh_rate` samples
                        if sample_count % refresh_rate == 0:
                            # Update time-domain plot
                            line1.set_ydata(list(B_data))
                            line1.set_xdata(list(time_data))

                            times = np.array(time_data)
                            voltages = np.array(B_data)

                            # Interpolate data for uniform sampling
                            interp_times = np.linspace(times[0], times[-1], len(times))
                            interp_voltages = interp1d(times, voltages, kind='linear')(interp_times)

                            # Calculate FFT
                            fft_values = np.fft.rfft(interp_voltages)
                            power_spectrum = np.abs(fft_values)**2
                            power_spectrum[power_spectrum == 0] = 1e-12  # Replace zeros for log scale
                            freqs = np.fft.rfftfreq(len(interp_times), d=(interp_times[1] - interp_times[0]))

                            # Update frequency-domain plot
                            fft_line.set_data(freqs, power_spectrum)
                            ax2.set_xlim(freqs[1], freqs.max())
                            ax2.set_ylim(1e-6, power_spectrum.max() * 1.1)

                            # Rescale and re-display the plots
                            ax1.relim()
                            ax1.autoscale_view()
                            ax2.relim()
                            ax2.autoscale_view()
                            clear_output(wait=True)
                            display(fig)

                    except (ValueError, IndexError):
                        # Skip lines that cannot be parsed
                        pass

        except (serial.SerialException, OSError):
            print("Device disconnected. Waiting for reconnection...")
            ser.close()
            ser = try_connect(serial_port, baud_rate)

except KeyboardInterrupt:
    print("Plotting stopped by user.")
finally:
    if 'ser' in locals() and ser.is_open:
        ser.close()


Reconnected to COM5
Device disconnected. Waiting for reconnection...
Waiting for device on COM5...
Waiting for device on COM5...
Waiting for device on COM5...
Reconnected to COM5
Plotting stopped by user.
Error in callback <function _draw_all_if_interactive at 0x0000018AE71F91F0> (for post_execute), with arguments args (),kwargs {}:


ValueError: Data has no positive values, and therefore cannot be log-scaled.

ValueError: Data has no positive values, and therefore cannot be log-scaled.

<Figure size 1000x800 with 2 Axes>